In [53]:
import os
import torch
import numpy as np
import pandas as pd
from RNARepLearn.utils import reconstruct_bpp
batch_path = "/lustre/groups/crna01/workspace/nicolas_msc/slurm/batch_log"
batch_size = 64
batches = os.listdir(batch_path) 

In [69]:
def extract_info(batch, batchsize):
    infos = []
    for i in range(batchsize):
        graph = batch[i]
        gc_content = sum(torch.logical_or(graph.x.argmax(dim=1)==1 , graph.x.argmax(dim=1)==2))/len(graph.x)
        bpp = reconstruct_bpp(graph.edge_index, graph.edge_weight, shape=(graph.x.shape[0], graph.x.shape[0]))
        info = {"length":graph.x.shape
                [0], "diag":sum(np.diagonal(bpp))/graph.x.shape[0], "bpp":bpp, "edge_index":graph.edge_index, "rfam":graph.rfam, "ID":graph.ID, "gc_content":gc_content.item()}
        infos.append(info)
    return infos
        

In [70]:
batch_info = []
for batch in batches:
    batch = torch.load(batch_path+"/"+batch, map_location=torch.device('cpu'))
    print(len(batch.ID))

    if len(batch)<64:
        continue

    batch_info.append(extract_info(batch, batch_size))

64
64
6
64
64
64
64
6
64
64
64
64
64
64
64
6
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64


In [71]:
from RNARepLearn.datasets import CombinedRfamDataset
from RNARepLearn.utils import train_val_test_loaders, add_backbone

ds = CombinedRfamDataset("/lustre/groups/crna01/workspace/nicolas_msc/RNARepLearn/rfam/data/raw/processed/release-14.8", None ,"All_300")# , add_LPE=True, add_backbone=True, transform=AddLaplacianEigenvectorPE(16, attr_name=None)
batch_size = 64

Processing complete


Processing...
Done!


In [72]:
train, val, test = train_val_test_loaders(ds, 0.001, 0.001, 0.998, 64, num_workers = 4)

Training:	695
Test:	694216
Validation:	697


In [73]:
it = iter(train)
test_batch_infos = []
for i in range(5):
    test_batch= next(it)
    test_info = extract_info(test_batch, batch_size)
    test_batch_infos.append(test_info)

In [74]:
import itertools
batch_info = pd.DataFrame(list(itertools.chain.from_iterable(batch_info)))
#
test_batch_info = pd.DataFrame(list(itertools.chain.from_iterable(test_batch_infos)))
#

In [75]:
batch_info

,length,diag,bpp,edge_index,rfam,ID,gc_content
0,58,0.428313,"[[0.9846336885287547, 0.0, 0.0, 0.0, 0.0001359...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF01704,AACY022116976.1_80-23,0.396552
1,55,0.419215,"[[0.7309638644110459, 0.0, 0.0, 0.0, 0.0, 4.23...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00066,CM003285.1_20125578-20125632,0.418182
2,113,0.392355,"[[0.018798724277636736, 0.0, 0.0, 0.0, 0.0, 0....","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00019,CM001942.1_47180701-47180589,0.433628
3,74,0.219056,"[[0.999308263315459, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF03506,CM000668.2_106784137-106784210,0.527027
4,139,0.331639,"[[0.9143254508358158, 0.0, 0.0, 0.0, 0.0, 1.01...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00321,KV778994.1_59132-59270,0.482014
...,...,...,...,...,...,...,...
1979,196,0.319797,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00004,CH963857.1_11928063-11927868,0.454082
1980,87,0.237512,"[[0.9962986119139073, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF01910,AGTP01041934.1_19370-19284,0.344828
1981,54,0.344409,"[[0.9813180345059394, 0.0, 0.0, 0.0, 4.4325255...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00436,CM008318.1_3718990-3718937,0.370370
1982,73,0.233752,"[[0.05174163978544788, 0.0, 0.0, 0.0, 0.0, 0.0...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00838,BABH01035687.1_2443-2371,0.479452


In [78]:
batch_info["type"]="batch"
test_batch_info["type"]="test"

In [79]:
batch_info

,length,diag,bpp,edge_index,rfam,ID,gc_content,type
0,58,0.428313,"[[0.9846336885287547, 0.0, 0.0, 0.0, 0.0001359...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF01704,AACY022116976.1_80-23,0.396552,batch
1,55,0.419215,"[[0.7309638644110459, 0.0, 0.0, 0.0, 0.0, 4.23...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00066,CM003285.1_20125578-20125632,0.418182,batch
2,113,0.392355,"[[0.018798724277636736, 0.0, 0.0, 0.0, 0.0, 0....","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00019,CM001942.1_47180701-47180589,0.433628,batch
3,74,0.219056,"[[0.999308263315459, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF03506,CM000668.2_106784137-106784210,0.527027,batch
4,139,0.331639,"[[0.9143254508358158, 0.0, 0.0, 0.0, 0.0, 1.01...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00321,KV778994.1_59132-59270,0.482014,batch
...,...,...,...,...,...,...,...,...
1979,196,0.319797,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00004,CH963857.1_11928063-11927868,0.454082,batch
1980,87,0.237512,"[[0.9962986119139073, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF01910,AGTP01041934.1_19370-19284,0.344828,batch
1981,54,0.344409,"[[0.9813180345059394, 0.0, 0.0, 0.0, 4.4325255...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00436,CM008318.1_3718990-3718937,0.370370,batch
1982,73,0.233752,"[[0.05174163978544788, 0.0, 0.0, 0.0, 0.0, 0.0...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...",RF00838,BABH01035687.1_2443-2371,0.479452,batch


In [82]:
info = pd.concat([batch_info,test_batch_info])

In [84]:
import seaborn as sns

plt.scatter(info["diag"], info["gc_content"], c=info["type"])

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
N = 50
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
area = (30 * np.random.rand(N))**2  # 0 to 15 point radii

plt.scatter(x, y, s=area, c=colors, alpha=0.5)
plt.show()